In [3]:
import pathlib

import gym
import matplotlib.pyplot as plt
from IPython.display import HTML, Markdown, display
from myst_nb import glue
from tabulate import tabulate

from src.utils import build_plots_dir_path
from src.visualization import PLOT_DPI

plt.ioff()  # turn off interactive plotting

root_dir = pathlib.Path().cwd().parent.parent.parent
cwd_dir_name = pathlib.Path().cwd().name

plot_dir = build_plots_dir_path(root_dir) / cwd_dir_name

(section-topics-environments)=
# Overview of the selected environments

This section describes six, both single- and multi-steps  environments used in further experiments. All of them are stationary toy-problems exhibiting real-valued properties facilitating the evaluation of modifications of anticipatory systems. Most of the environments were used as benchmarks in other works related to measuring the performance of the LCS, however the [](section-topics-environments-cartpole) environment was never used in this class of algorithms before.

````{margin}
```{admonition} Stationary environment
A stationary environment refers to data-generating distributions that **do not change over time**.
```
````

All the mentioned environments poses the standardized interface for enabling the agent to interact with it in consistent manner. In each step they present the current observation (that is not equal to its internal state), possible reward from previously executed action and the information whether the interaction is finished. In most cases there is a certain reward state, that acts as an incentive, motivating the forager to reach it.

(section-topics-environments-corridor)=
## Corridor
The corridor is a 1D multi-step, linear environment introduced by Lanzi to evaluate the XCSF agent {cite}`lanzi2005xcs`. The system output is defined over a finite discrete interval $[0,n]$. On each trial, the agent is placed randomly on the path and can execute two possible actions - move left or right (which corresponds to moving one unit in a particular direction - see Figure {numref}`{number} <corridor-env>`.

:::{figure-md} corridor-env
<img src="../../_static/corridor.png" width="75%">

The _Corridor_ environment. The size of the input space is $n$.
:::

Lanzi used a real-valued version of this environment where the agent location is elucidated by a value between $[0,1]$. Predefined step size was added to the current position when it executed an action, thus changing its value. When the agent reaches the final state $s = 1.0$, the reward is paid out.

The environment examined herein signifies the state already in discretized form, available in three preconfigured sizes with increasing difficulty. The main challenge for the agent here is mainly to successfully learn the reward distribution in possibly long action chains.

**Reward scheme**: The trial ends when it reaches the final state $n$ (obtaining reward $r=1000$) or when the maximum number of 200 steps in each episode is exceeded. Otherwise, the reward after each step is $r=0$.

(section-topics-environments-grid)=
## Grid
Grid refers to an extension of the [](section-topics-environments-corridor) environment {cite}`lanzi2005xcs`. A vertical dimension and two new actions (move up, move down) are added. The raw agent perception is now identified as a pair of real numbers $(s_0, s_1)$, where $s \in [0,1]$. Similarly, the environment is presented to the agent in a discretized form. Each dimension is divided into $n-1$ equally spaced buckets - see Figure {numref}`{number} <grid-env>`.

:::{figure-md} grid-env
<img src="../../_static/grid.png" width="75%">

The _Grid_ environment. The size of the input space is $n^2$. The agent can change its position by moving in four different directions.
:::

**Reward scheme**: The trial ends with reward $r=1000$ when the final state $(n, n)$ is reached. Otherwise, the reward after each step is $r=0$. Additionally, the episode is terminated after exceeding the maximum number of 2000 steps.

(section-topics-environments-rmpx)=
## Real Multiplexer (rMPX)

In [ ]:
import gym_multiplexer  # noqa: F401

rmpx6bit_env = gym.make('real-multiplexer-6bit-v0')
random_state = rmpx6bit_env.reset()

threshold = 0.5  # secret knowledge

t = HTML(tabulate([
    ['<b>rMPX signal</b>'] + [round(s, 2) for s in random_state],
    ['<b>MPX signal </b>'] + list(map(lambda x: 1 if x > threshold else 0, random_state))
], tablefmt='unsafehtml', stralign='right'))

glue('rmpx_mpx_mapping_table', t, display=False)

The modification to the traditional Boolean Multiplexer (MPX) was introduced by Wilson {cite}`wilson1999get` to examine the performance in single-step environments using real-valued data.

````{margin}
```{admonition} Epistasis and heterogeneity
**Epistasis** describes an non-linear interaction effect between multiple features.

**Heterogenity** means that for different sets of instances, a distinct subset of features will determine the class value.
```
````

The Boolean _n_-bit multiplexer defines a set of single-step supervised learning problems that are conceptually based on an electronic device taking multiple inputs and switching them to the single output. In each trial a random, fixed binary string of predefined length is generated. It comprises two parts - the address and register segments. The first one points to specific register address that is considered to be a truth value - Figure {numref}`{number} <mpx6bit>` describes the process. This environment is also particularly interesting because it possesses the properties of _epistasis_  and _heterogeneity_, which are often present in within real-world problems such as bioinformatics, finance or behaviour modelling.

:::{figure-md} mpx6bit
<img src="../../_static/mpx6bit.png" width="75%">

Visualization of determining the output value of 6 bit multiplexer. The address bit points to the first value of the register array that is considered the true output. Diagram taken from {cite}`urbanowicz2017introduction`.
:::

For the rMPX the only difference between boolean multiplexer is that generated perception consists of real-values drawn from a uniform distribution. To validate the correct answer, the additional variable - secret threshold $\theta = 0.5$ is used to map each allele into binary form. See Table {numref}`{number} <rmpx_mpx_mapping_table>` for the example of such mapping.

```{glue:figure} rmpx_mpx_mapping_table
:align: center
:figwidth: 500px
:name: "rmpx_mpx_mapping_table"

Example of mapping a random 6-bit rMPX signal into MPX problem. A threshold of $\theta=0.5$ was used. The last bit is set to $0$ as an initial value that will be changed to $1$ after performing the correct action.
```

The standard version, however, is still not suitable to be used with ALCS. Because an agent utilizes perceptual causality to form new classifiers, assuming that after executing an action, the state will change. The MPX does not have any possibility to send feedback about the correctness of the action. Butz suggested two solutions to this problem {cite}`butz2002anticipatory`. In this work, the assumption is that the state generated by the rMPX is extended by one extra bit, denoting whether the classification was successful. This bit is by default set to zero the agent responds correctly after being switched, thus providing a direct feedback. A detailed example can be found in {cite}`kozlowski2019preliminary`.

**Reward scheme**: If the correct answer is given the reward $r=1000$ is paid out, otherwise $r=0$.

(section-topics-environments-checkerboard)=
## Checkerboard

In [2]:
def plot_checkerboard(plot_filename=None):
    import gym_checkerboard  # noqa: F401
    checkerboard_env = gym.make('checkerboard-2D-3div-v0')
    checkerboard_env.reset()

    np_board = checkerboard_env.env._board.board

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)

    ax.matshow(np_board, cmap=plt.get_cmap('gray_r'), extent=(0, 1, 0, 1), alpha=.5)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI)

    return fig

glue('checkerboard-env', plot_checkerboard(f'{plot_dir}/checkerboard-env-visualization.png'), display=False)

NameError: name 'pylab' is not defined

The Checkerboard is a single-step environment introduced by Stone in {cite}`stone2003real`. It was proposed to circumvent certain limitations of the [](section-topics-environments-rmpx) when using the interval predicates approach. Because the rMPX problem can be solved by using just a hyperplane decision surface, it's not considered of being able to represent arbitrary intervals in solution space. In order to solve the Checkerboard problem a hyper-rectangle decision surface is needed for modelling certain interval regions.

This environment works by dividing up the $n$-dimensional solution space into equal-sized hypercubes. Each hypercube is assigned a white or black color (alternating in all dimensions), see Figure {numref}`{number} <checkerboard-env>`. The problem difficulty can be controlled by changing both the dimensionality $n$ and the number of divisions in each dimension $n_d$. In order to allow the colors to be alternating $n_d$ must be an odd number.

```{glue:figure} checkerboard-env
:name: "checkerboard-env"
2-dimensional Checkerboard problem with $n_d=3$. Precise interval predicates are needed for representing selected regions.
```
In each trial the environment presents a vector of length $n_d$ or real numbers in the interval $[0, 1)$, representing a point in the solution space. The agent's goal is to guess the correct color by performing one of two actions depending on the colour (black or white) of a pointed hypercube.

**Reward scheme**: When the correct answer is given the reward $r=1$ is paid out, otherwise $r=0$.

(section-topics-environments-cartpole)=
## Cart Pole

:::{figure-md} cartpole-env
<img src="../../_static/cartpole.gif">

The cartpole environment
:::

(section-topics-environments-fsw)=
## Finite State Worlds (FSW)
...

:::{figure-md} fsw-env
<img src="../../_static/fsw.png">

A Finite State World of length 5 (FSW-5). Environment is especially suited for measuring a reward propagation for long action chains.
:::

(section-topics-environments-woods1)=
## Woods1
Why this environment was considered in this work?